In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

In [2]:
tp_df = pd.read_excel("SimData_Transplant_Ver1.0.xls")
tp_df

,Gender,Bloodtype,Inpt_attx,Donor_Sex,Donor_Bloodtype,Donor_DCD,Waittime,Age_Tx,Donor_Age,TransplantBMI,Donor_BMI,MELD,MELDNA,Time,Cens
0,Male,A,home,Male,A,No,268,56.556760,41.002731,33.445563,31.358900,13.967938,14.861206,1770.322146,1
1,Male,A,home,Male,A,Yes,39,60.818110,30.997162,19.928849,22.484298,12.414717,14.542220,1432.093391,1
2,Male,AB,home,Male,A,No,511,51.127335,33.616883,27.273124,24.887090,7.680005,7.684472,815.728909,1
3,Male,B,home,Male,B,No,126,53.782981,39.580918,27.275657,33.181029,20.663905,21.302559,408.557233,1
4,Male,O,icu,Male,O,No,7,78.053001,51.286678,23.349091,28.348204,25.454707,24.021749,4676.209514,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,Male,A,home,Female,A,No,179,66.435868,30.960044,24.850272,19.682428,12.620159,9.912282,4658.526132,1
3996,Male,O,inpt,Female,A,No,25,52.845865,71.273963,29.179275,32.437191,29.698726,29.692512,505.425267,1
3997,Male,A,inpt,Female,O,No,153,34.176371,28.217301,33.598114,23.537730,27.098878,29.459862,307.379955,1
3998,Female,O,home,Male,O,No,12,59.653318,36.775017,24.560432,18.579377,22.864374,24.288275,145.425581,0


In [3]:
wl_df = pd.read_excel("SimData_WList_Ver1.0.xls")
wl_df

,Gender,Bloodtype,Inpt_attx,Age,BMI,MELD,MELDNA,Time,Cens
0,Female,B,home,61.085467,24.339517,10.655185,10.655894,1941.074915,1
1,Male,B,home,61.489707,27.424602,14.165872,12.658377,4.610223,1
2,Male,A,home,38.261000,25.562109,17.921970,26.669315,1108.553992,1
3,Male,O,inpt,36.191386,33.157785,34.993244,34.481114,1228.591710,1
4,Male,B,inpt,59.528581,26.426496,21.157155,19.292050,5981.142742,1
...,...,...,...,...,...,...,...,...,...
4995,Male,O,ventilated,18.452599,26.009412,33.244652,32.888834,6.180853,0
4996,Male,O,home,64.744593,26.688906,8.674914,8.674934,1152.335589,1
4997,Male,O,home,46.251465,32.578422,16.535037,19.861037,366.774015,1
4998,Male,AB,home,55.561647,24.147084,25.891168,26.378647,296.153015,0


In [4]:
test_df = pd.get_dummies(wl_df) 
test_df = test_df.drop(['Cens', 'Time'], axis=1)
for i in test_df:
    print(i)
test_df

Age
BMI
MELD
MELDNA
Gender_Female
Gender_Male
Bloodtype_A
Bloodtype_AB
Bloodtype_B
Bloodtype_O
Inpt_attx_home
Inpt_attx_icu
Inpt_attx_inpt
Inpt_attx_ventilated


,Age,BMI,MELD,MELDNA,Gender_Female,Gender_Male,Bloodtype_A,Bloodtype_AB,Bloodtype_B,Bloodtype_O,Inpt_attx_home,Inpt_attx_icu,Inpt_attx_inpt,Inpt_attx_ventilated
0,61.085467,24.339517,10.655185,10.655894,1,0,0,0,1,0,1,0,0,0
1,61.489707,27.424602,14.165872,12.658377,0,1,0,0,1,0,1,0,0,0
2,38.261000,25.562109,17.921970,26.669315,0,1,1,0,0,0,1,0,0,0
3,36.191386,33.157785,34.993244,34.481114,0,1,0,0,0,1,0,0,1,0
4,59.528581,26.426496,21.157155,19.292050,0,1,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,18.452599,26.009412,33.244652,32.888834,0,1,0,0,0,1,0,0,0,1
4996,64.744593,26.688906,8.674914,8.674934,0,1,0,0,0,1,1,0,0,0
4997,46.251465,32.578422,16.535037,19.861037,0,1,0,0,0,1,1,0,0,0
4998,55.561647,24.147084,25.891168,26.378647,0,1,0,1,0,0,1,0,0,0


In [5]:
from sklearn import preprocessing
from copy import deepcopy
from sklearn.preprocessing import StandardScaler
df = deepcopy(tp_df)
num_df = pd.get_dummies(df)
num_df = num_df.drop(['Waittime','Donor_Age','Donor_BMI','Donor_Sex_Female',\
                  'Donor_Sex_Male','Donor_Bloodtype_A', 'Donor_Bloodtype_B', 'Donor_Bloodtype_O',\
                  'Donor_Bloodtype_AB', 'Donor_DCD_No', 'Donor_DCD_Yes'], axis=1)
print(num_df)
#mean = np.mean(np.array(num_df['Time']))
#std = np.std(np.array(num_df['Time']))
#normal_array = (np.array(num_df['Time']) - mean)/ std
#print(normal_array)

train_data = preprocessing.StandardScaler().fit_transform(num_df)
train_df = pd.DataFrame(train_data, columns=['A', 'B', 'C', 'D','Time',  'H', 'I', 'J', 'K', 'L', 'M', 'N'
                                            'O','P','Q','R', "S"]) 

labels = np.array(train_df['Time'])

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_df, labels,train_size=0.8,random_state= 42)


print(X_train)

         Age_Tx  TransplantBMI       MELD     MELDNA         Time  Cens  \
0     56.556760      33.445563  13.967938  14.861206  1770.322146     1   
1     60.818110      19.928849  12.414717  14.542220  1432.093391     1   
2     51.127335      27.273124   7.680005   7.684472   815.728909     1   
3     53.782981      27.275657  20.663905  21.302559   408.557233     1   
4     78.053001      23.349091  25.454707  24.021749  4676.209514     1   
...         ...            ...        ...        ...          ...   ...   
3995  66.435868      24.850272  12.620159   9.912282  4658.526132     1   
3996  52.845865      29.179275  29.698726  29.692512   505.425267     1   
3997  34.176371      33.598114  27.098878  29.459862   307.379955     1   
3998  59.653318      24.560432  22.864374  24.288275   145.425581     0   
3999  38.954687      18.577734   9.315963   9.941745  3751.267686     1   

      Gender_Female  Gender_Male  Bloodtype_A  Bloodtype_AB  Bloodtype_B  \
0                 0    

In [6]:
# importing Pool
from catboost import Pool
#Creating pool object for train dataset. we give information of categorical fetures to parameter cat_fetaures
train_pool = Pool(data=X_train,
                  label=y_train
                 )
#Creating pool object for test dataset
test_pool = Pool(data=X_test,
                  label=y_test
                 )

In [7]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split

In [8]:
model = CatBoostRegressor(iterations=10000,
                          loss_function='RMSE',
                          eval_metric='RMSE',
                          metric_period=50
                         )

In [9]:
model.fit(train_pool, eval_set=test_pool)

Learning rate set to 0.015004
0:	learn: 0.9889546	test: 0.9772370	best: 0.9772370 (0)	total: 56.9ms	remaining: 9m 28s
50:	learn: 0.5196017	test: 0.5242258	best: 0.5242258 (50)	total: 134ms	remaining: 26.2s
100:	learn: 0.2787990	test: 0.2859119	best: 0.2859119 (100)	total: 176ms	remaining: 17.2s
150:	learn: 0.1558639	test: 0.1614191	best: 0.1614191 (150)	total: 222ms	remaining: 14.5s
200:	learn: 0.0920781	test: 0.0975026	best: 0.0975026 (200)	total: 273ms	remaining: 13.3s
250:	learn: 0.0606267	test: 0.0657975	best: 0.0657975 (250)	total: 325ms	remaining: 12.6s
300:	learn: 0.0449903	test: 0.0502520	best: 0.0502520 (300)	total: 375ms	remaining: 12.1s
350:	learn: 0.0373090	test: 0.0437745	best: 0.0437745 (350)	total: 423ms	remaining: 11.6s
400:	learn: 0.0320548	test: 0.0395585	best: 0.0395585 (400)	total: 473ms	remaining: 11.3s
450:	learn: 0.0283130	test: 0.0369325	best: 0.0369325 (450)	total: 524ms	remaining: 11.1s
500:	learn: 0.0253762	test: 0.0349900	best: 0.0349900 (500)	total: 583ms	r

4500:	learn: 0.0047454	test: 0.0259831	best: 0.0259813 (4450)	total: 5.74s	remaining: 7.01s
4550:	learn: 0.0047057	test: 0.0259793	best: 0.0259793 (4550)	total: 5.8s	remaining: 6.95s
4600:	learn: 0.0046674	test: 0.0259800	best: 0.0259793 (4550)	total: 5.87s	remaining: 6.89s
4650:	learn: 0.0046245	test: 0.0259786	best: 0.0259786 (4650)	total: 5.93s	remaining: 6.82s
4700:	learn: 0.0045815	test: 0.0259802	best: 0.0259786 (4650)	total: 6s	remaining: 6.76s
4750:	learn: 0.0045425	test: 0.0259865	best: 0.0259786 (4650)	total: 6.06s	remaining: 6.7s
4800:	learn: 0.0045063	test: 0.0259917	best: 0.0259786 (4650)	total: 6.12s	remaining: 6.63s
4850:	learn: 0.0044689	test: 0.0259938	best: 0.0259786 (4650)	total: 6.18s	remaining: 6.56s
4900:	learn: 0.0044319	test: 0.0259871	best: 0.0259786 (4650)	total: 6.24s	remaining: 6.5s
4950:	learn: 0.0043947	test: 0.0259881	best: 0.0259786 (4650)	total: 6.3s	remaining: 6.43s
5000:	learn: 0.0043565	test: 0.0259807	best: 0.0259786 (4650)	total: 6.37s	remaining: 6

9100:	learn: 0.0025412	test: 0.0259352	best: 0.0259246 (8200)	total: 11.8s	remaining: 1.17s
9150:	learn: 0.0025288	test: 0.0259346	best: 0.0259246 (8200)	total: 11.9s	remaining: 1.11s
9200:	learn: 0.0025142	test: 0.0259348	best: 0.0259246 (8200)	total: 12s	remaining: 1.04s
9250:	learn: 0.0025008	test: 0.0259326	best: 0.0259246 (8200)	total: 12.1s	remaining: 979ms
9300:	learn: 0.0024890	test: 0.0259316	best: 0.0259246 (8200)	total: 12.2s	remaining: 914ms
9350:	learn: 0.0024750	test: 0.0259325	best: 0.0259246 (8200)	total: 12.2s	remaining: 849ms
9400:	learn: 0.0024624	test: 0.0259322	best: 0.0259246 (8200)	total: 12.3s	remaining: 783ms
9450:	learn: 0.0024485	test: 0.0259336	best: 0.0259246 (8200)	total: 12.4s	remaining: 718ms
9500:	learn: 0.0024352	test: 0.0259338	best: 0.0259246 (8200)	total: 12.4s	remaining: 652ms
9550:	learn: 0.0024217	test: 0.0259325	best: 0.0259246 (8200)	total: 12.5s	remaining: 586ms
9600:	learn: 0.0024092	test: 0.0259315	best: 0.0259246 (8200)	total: 12.5s	remaini

In [10]:
from lifelines.utils import concordance_index
from sklearn.metrics import mean_squared_error

In [11]:
train_y_pred = model.predict(train_pool)
test_y_pred = model.predict(test_pool)

print("RMSE",np.sqrt(mean_squared_error(y_train, train_y_pred)))
print("RMSE",np.sqrt(mean_squared_error(y_test, test_y_pred)))

RMSE 0.0028192433201074587
RMSE 0.025924550621771632


In [12]:
for i in train_y_pred:
    print(i)

-0.3731146277842082
-0.8946030901807509
2.0075783082671332
-0.6568570675870984
5.326039201989253
0.24330222428601112
-0.4171069746294889
0.005831799031455136
0.5711081147721467
0.5912894358618481
2.4167408329247975
-0.9045386544042613
-0.4898266437419721
-0.5865739913992919
1.137180070429292
0.08960451141217847
-0.17220460152607034
-0.6746851381400365
3.212559904681405
1.1578472577390022
1.2018599146489717
-0.9516709918242432
-0.6939030663201798
2.505965995167717
1.85418080531024
-0.8018563728952308
-0.3560277526930977
-0.5408797103064203
-0.9895505888084676
-0.629296252003582
-0.5108593968437549
-0.976400381961171
0.7016612651762988
3.733111140411751
0.9834557661789259
-0.9446740280189739
-0.7811351424297207
-0.30832696838517426
-0.35932580903271455
-0.12264119055560888
-0.958526198739433
-0.267846424911976
-0.5625156235413521
0.9300309843243213
0.8957675099760786
-0.34409892141306314
-0.2549976961556422
-0.877394190213524
-0.035080947756561384
0.7972221106014238
0.016177605849614335


-0.7252624706738443
-0.706101803218651
0.005550176956086717
0.06234263938129539
-0.8070292439179451
1.6159079899997635
-0.006361010314840573
1.5090898518056122
-0.712256989976719
0.4105564899457497
-0.8671850421505988
0.5320336826274338
0.6133659695072928
-0.7183862576533397
-0.6457999918757383
-0.3916458237382426
-0.7570444617047482
0.7640325045468059
-0.6795287797437113
-0.8250850671950785
-0.7458525372319184
0.9515111407129584
-0.26931684602091654
-0.5690159720400711
0.6075842129500016
-0.8848109521152037
-0.5670802202581776
-0.7779274207162648
-0.6935446032848973
3.3182215093906247
1.3901844560970074
4.8091419336198165
-0.27607667391602003
-0.6582514470879227
0.2091098762437419
0.3720846401035938
2.742375860596007
-0.6132494183479448
0.6401044356498978
0.1480344699817143
0.15634897292012465
0.3558573575648216
0.35546451808548507
-0.7014779686037252
1.1268514391226172
0.7797862178977135
2.122655428778984
-0.2768575737173615
-0.42501253274654605
0.07730229325597286
-0.621148658489629

In [13]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
print("RMSE",np.sqrt(mean_squared_error(y_test, test_y_pred)))
print("MAE",mean_absolute_error(y_test, test_y_pred))
print("R2", r2_score(y_test, test_y_pred, force_finite=False))

RMSE 0.025924550621771632
MAE 0.008008915733555326
R2 0.9993143639567469
